<div style = "text_algin: center; font-size: 40px; font-weight: bold">
Phase 01: Collecting data
</div>

## Import necessary Python modules

In [1]:
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
import re

We will collect to type of data: The real estate for sale and the real eastate for rent

## Part 01: Collect real estate for sale data:

### 1. Initials:

Create an empty dataframe to store the data of the real estate. The dataframe will contain some features like: Address, Area, Price, Bedroom, Toilet, Floor, Furniture, Direction, Legal, Posting date, Expiry date, Ad type, Ad code

In [2]:
fields = [ 'Address', 'Area', 'Price', 'Bedroom', 'Toilet', 'Floor', 'Furniture', 'Direction','Legal', 'Posting date', 'Expiry date', 'Ad type', 'Ad code']

df = pd.DataFrame(columns = fields)

(0, 13)


### 2. Collect the links to each real estate page:

The links are collected from multiple pages, with each page containing numerous links to apartments. These links have been stored in a file named "real_estate_for_sale_links.txt".     
The link to the homepage and the file name are provided below:

In [3]:
file_name = "real_estate_for_sale_links.txt"
url = 'https://batdongsan.com.vn/nha-dat-ban-tp-hcm'

We will move from the first page to the last page to collect the links, so the first thing should do is get the final page of the website.

In [4]:
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

unprocessed_num_page = soup.find(class_ = 're__pagination-group')
driver.quit()

NameError: name 'url' is not defined

Process the result to get the last page.

In [ ]:

regex = r'[^\s]+'
matchs = re.findall(regex, unprocessed_num_page.text)
new_array = []
for match in matchs:
    new_array.append(int(match.replace('.', '')))

total_page = max(new_array)

Function to collect the links to each real estate:

In [ ]:
def CollectRealEstateForSaleLink (url, page):
    with open (file_name, 'w') as file:

        for i in range(1, page + 1):
            driver = webdriver.Chrome()
            driver.get(url+'/p'+ str(i))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            apartment_list = soup.find_all("a", class_= "js__product-link-for-product-id")

            for apartment in apartment_list:
                link = 'https://batdongsan.com.vn/'+ apartment['href']
                file.write(link)
                file.write('\n')

            driver.quit()

In [ ]:
CollectRealEstateForSaleLink (url, total_page)

Read the "real_estate_for_sale_links.txt" file and store all the links to a lists

In [ ]:
urls = []
with open(file_name, 'r') as file:
    for line in file.readlines():
            urls.append(line)

### 3. Access to each link and colllect the data:

- A dictionary to rename the features collected from the webiste:

In [4]:
key_map = {
    'Địa chỉ:' : 'Address',
    'Diện tích': 'Area',
    'Mức giá': 'Price',
    'Hướng nhà': 'Direction',
    'Số tầng': 'Floor',
    'Số phòng ngủ': 'Bedroom',
    'Số toilet': 'Toilet',
    'Pháp lý': 'Legal',
    'Nội thất': 'Furniture',
    'Ngày đăng': 'Posting date',
    'Ngày hết hạn': 'Expiry date',
    'Loại tin': 'Ad type',
    'Mã tin': 'Ad code'
}

- Function to extract the features from a link:

In [5]:
def ExtractDataFromLink(url):
    driver = webdriver.Chrome()
    try:
        driver.get(url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        #Create a dicitonary to store the feature
        features = {}

        # real_estate_list = soup.find_all("a", class_= "js__product-link-for-product-id")
        # if real_estate_list:
        #     for item in real_estate_list:
        #         link = 'https://batdongsan.com.vn/'+ apartment['href']
        #         features = ExtractDataFromLink(link)
        # else:

        # get the address
        address = soup.find(class_ = "re__pr-short-description js__pr-address")

        if address == None:
            return features
        else:
            features['Address'] = address.text

        # Get all the features
        feature_items = soup.find_all('div', class_ = 're__pr-specs-content-item')
        for item in feature_items:
            title = item.find(class_ = 're__pr-specs-content-item-title').text.strip()
            value = item.find(class_ = 're__pr-specs-content-item-value').text.strip()
            features[title] = value

        date_items = soup.find_all( 'div', class_ = "re__pr-short-info-item js__pr-config-item")
        for item in date_items:
            title = item.find('span', class_ = 'title').text.strip()
            value = item.find('span',class_ = 'value').text.strip()
            features[title] = value
    
        keys_to_change = list(features.keys())
        for key in keys_to_change:
            if key in key_map:
                features[key_map[key]] = features[key]
    finally:
        driver.quit()

    return features

Iterate over the urls list and extract the data, which is stored in the previously defined DataFrame

In [15]:
for url in urls[ 564 : 1000]:
        features = ExtractDataFromLink(url)
        if features:
                new_row = pd.Series(features, index = fields)
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df], ignore_index = True)
                
                new_row_df.to_csv('real_estate_for_sale.csv', mode='a', header=False, index=False, encoding='utf-8')
        else:
                print ("hihi")


In [5]:
with open (file_name, 'a') as file:
    for i in range(1301, 1400 + 1):
        driver = webdriver.Chrome()
        driver.get(url+'/p'+ str(i))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        apartment_list = soup.find_all("a", class_= "js__product-link-for-product-id")

        for apartment in apartment_list:
            link = 'https://batdongsan.com.vn/'+ apartment['href']
            file.write(link)
            file.write('\n')

        driver.quit()

### 4. Store the collected data to csv file:

In [21]:
df.to_csv('real_estate_for_sale.csv', index = False)

In [10]:
real_estate_df = pd.read_csv('real_estate_for_sale.csv')

In [12]:
real_estate_df.shape()

,Address,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",77 m²,"6,1 tỷ",2 phòng,2 phòng,NaN,NaN,NaN,NaN,27/11/2023,07/12/2023,Tin VIP Kim Cương,38645401
1,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",76 m²,"6,2 tỷ",2 phòng,2 phòng,NaN,Đầy đủ,NaN,Hợp đồng mua bán,23/11/2023,30/11/2023,Tin thường,38393009
2,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",72 m²,Thỏa thuận,2 phòng,2 phòng,NaN,NaN,NaN,NaN,28/11/2023,05/12/2023,Tin VIP Kim Cương,38657478
3,"Lumiere Riverside, 628, Đường Xa Lộ Hà Nội, Ph...","77,1 m²","6,6 tỷ",2 phòng,2 phòng,NaN,Cơ bản,NaN,Hợp đồng mua bán,29/11/2023,06/12/2023,Tin VIP Kim Cương,38403632
4,"Dự án The Privia, Đường An Dương Vương, Phường...","67,74 m²","3,5 tỷ",2 phòng,2 phòng,NaN,"Bàn giao cơ bản các thiết bị Châu Âu cao cấp, ...",NaN,HĐMB,29/11/2023,06/12/2023,Tin VIP Kim Cương,38654457
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Đường Nhuận Đức, Xã Nhuận Đức, Củ Chi, Hồ Chí ...",4.250 m²,"6,3 triệu/m²",NaN,NaN,NaN,NaN,NaN,Sổ hồng riêng,28/11/2023,13/12/2023,Tin VIP Vàng,38451882
94,"Dự án Akari City Nam Long, Đường Võ Văn Kiệt, ...",60 m²,Thỏa thuận,2 phòng,1 phòng,NaN,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,28/11/2023,05/12/2023,Tin VIP Vàng,38653484
95,"Dự án HaDo Centrosa Garden, Đường 3/2, Phường ...",86 m²,"6,6 tỷ",2 phòng,2 phòng,NaN,NaN,NaN,NaN,23/11/2023,08/12/2023,Tin VIP Vàng,35489279
96,"Dự án KDC Phú Mỹ, Đường Hoàng Quốc Việt, Phườn...",126 m²,"20,7 tỷ",4 phòng,5 phòng,3 tầng,NaN,NaN,Có sổ,28/11/2023,08/12/2023,Tin VIP Vàng,38526845


## Part 02: Collect real estate for rent data:


# Finish collecting data